In [194]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

import time

In [195]:
wine_df = pd.read_csv("data/winequalityN.csv")
wine_df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [196]:
cleanup_wines = {"type":     {"white": 0, "red": 1}
                }
wine_df.replace(cleanup_wines,inplace = True)
wine_df = wine_df.dropna()


In [197]:
target = wine_df['quality']
target[target < 6] = 0
target[target >= 6] = 1
wine_df.drop( ['quality'],axis = 1,inplace = True)

/anaconda3/envs/mlclass/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [198]:
wine_df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8
1,0,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5
2,0,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1
3,0,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9
4,0,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9


In [199]:
X_train,X_test,Y_train,Y_test = train_test_split(wine_df, target, test_size=0.4, random_state = 18)

In [200]:
print(len(X_train))
print(len(Y_train))
print(len(X_test))
print(len(Y_test))

3877
3877
2586
2586


In [201]:
target.value_counts()

1    4091
0    2372
Name: quality, dtype: int64

In [202]:
sc=StandardScaler()
sc.fit(X_train)
X_train_std=sc.transform(X_train)
X_test_std=sc.transform(X_test)

In [203]:
from sklearn import tree
model = tree.DecisionTreeClassifier()
model.fit(X_train,Y_train)
model.score(X_train,Y_train)

1.0

In [204]:
clf_dt = tree.DecisionTreeClassifier(random_state=7)
clf_dt.fit(X_train, Y_train)
y_pred = clf_dt.predict(X_test)
dt_accuracy = accuracy_score(Y_test, y_pred)
print('Accuracy of decision tree without hyperparameter tuning is %.2f%%' % (dt_accuracy*100))

Accuracy of decision tree without hyperparameter tuning is 75.37%


In [205]:
from sklearn.neighbors import KNeighborsClassifier
model2=KNeighborsClassifier(n_neighbors=10)
model2.fit(X_train,Y_train)
model2.score(X_train,Y_train)

0.7384575702863039

In [206]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(hidden_layer_sizes=(1, 1), random_state=42, max_iter=1000)
nn.fit(X_train_std,Y_train)
nn.score(X_train,Y_train)

0.6355429455764766

In [207]:
from sklearn import svm
from sklearn.svm import SVC
svc = svm.SVC(kernel='rbf')
svc.fit(X_train_std,Y_train)
svc.score(X_train_std, Y_train)

0.7905597111168429

In [208]:
dt_stump = tree.DecisionTreeClassifier(max_depth=1, min_samples_leaf=1)
clf_boosted = AdaBoostClassifier(base_estimator=dt_stump, random_state=42)
clf_boosted.fit(X_train, Y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=50, random_state=42)

In [209]:
y_pred=nn.predict(X_test_std)
print('Misclassified samples using Perceptron are: %d' %(Y_test!=y_pred).sum())
print('Classification Accuracy of Perceptron is %.2f ' %accuracy_score(Y_test,y_pred))

Misclassified samples using Perceptron are: 959
Classification Accuracy of Perceptron is 0.63 


In [210]:
s_pred=svc.predict(X_test_std)
print('Misclassified samples using SVM are: %d' %(Y_test!=s_pred).sum())
print('Classification Accuracy of SVM is %.2f ' %accuracy_score(Y_test,s_pred))

Misclassified samples using SVM are: 602
Classification Accuracy of SVM is 0.77 


In [211]:
t_pred= model.predict(X_test)
print('Misclassified samples using Trees are: %d' %(Y_test!=t_pred).sum())
print('Classification Accuracy of Decision trees is %.2f ' %accuracy_score(Y_test,t_pred))

Misclassified samples using Trees are: 650
Classification Accuracy of Decision trees is 0.75 


In [212]:
k_pred= model2.predict(X_test)
print('Misclassified samples using KNN are: %d' %(Y_test!=k_pred).sum())
print('Classification Accuracy of KNN is %.2f ' %accuracy_score(Y_test,k_pred))

Misclassified samples using KNN are: 869
Classification Accuracy of KNN is 0.66 


In [213]:
f_pred = clf_boosted.predict(X_test)
print('Misclassified samples using Boost are: %d' %(Y_test!=f_pred).sum())
print('Classification Accuracy of Boost is %.2f ' %accuracy_score(Y_test,f_pred))

Misclassified samples using Boost are: 619
Classification Accuracy of Boost is 0.76 
